In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
import wandb
from tqdm.auto import tqdm

sys.path.append("..")
from otc.features.build_features import features_classical_size
from otc.models.classical_classifier import ClassicalClassifier


In [ ]:
# set here globally
seed = 42

exchange = "cboe" # "ise"
models = "classical"
subset = "all" # "test"
strategy = "supervised" # "transfer"


In [ ]:
# key used for files and artefacts
key = f"{exchange}_{models}_{strategy}_{subset}"

dataset = f"fbv/thesis/{exchange}_{strategy}_none:latest"


In [ ]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"
run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(dataset)
data_dir = artifact.download()


In [ ]:
columns = [
    *features_classical_size,
    "buy_sell",
]


In [ ]:
if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set.parquet"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set.parquet"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set.parquet"), engine="fastparquet", columns=columns
    )
    data = pd.concat([train, val, test])
    del train, val, test

elif subset == "test":
    data = pd.read_parquet(
        Path(data_dir, "test_set.parquet"), engine="fastparquet", columns=columns
    )

y_test = data["buy_sell"].astype("int8")
X_test = data.drop(columns="buy_sell")

del data


In [ ]:
# rules
rules = [
    [("tick", "all")],
    [("tick", "ex")],
    [("quote", "best")],
    [("quote", "ex")],
    [("lr", "ex")],
    [("lr", "best")],
    [("rev_lr", "ex")],
    [("rev_lr", "best")],
    [("emo", "ex")],
    [("emo", "best")],
    [("rev_emo", "ex")],
    [("rev_emo", "best")],
    [("clnv", "ex")],
    [("clnv", "best")],
    [("rev_clnv", "ex")],
    [("rev_clnv", "best")],
    [("trade_size", "ex"), ("tick", "all")],
    [("trade_size", "ex"), ("quote", "best")],
    [
        ("trade_size", "ex"),
        ("quote", "best"),
        ("quote", "ex"),
    ],  # classical + trade size
    [("quote", "best"), ("quote", "ex")],  # murjajev
    [
        ("trade_size", "ex"),
        ("quote", "best"),
        ("quote", "ex"),
        ("depth", "best"),
        ("depth", "ex"),
        ("rev_tick", "all"),
    ],  # p. 13 grauer
]

# generate names for array
names = []
for r in tqdm(rules):
    name = "->".join("%s(%s)" % tup for tup in r)
    names.append(name)


In [ ]:
names


In [ ]:
rules


In [ ]:
results = []

for rule in tqdm(rules):
    clf = ClassicalClassifier(layers=rule, random_state=seed, strategy="const")
    # fit is only used to set sklearn attributes, no leakage
    clf.fit(X=X_test.head(5), y=y_test.head(5))
    result = clf.predict(X_test)
    results.append(result)


In [ ]:
results = pd.DataFrame(dict(zip(names, results)), index=X_test.index)


In [ ]:
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/results/{key}.parquet"
)
results.to_parquet(output_path)


In [ ]:
# Log the artifact to save it as an output of this run
result_set = wandb.Artifact(name=key, type="results")
result_set.add_reference(output_path, name="results")
run.log_artifact(result_set)

wandb.finish()
